In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

C:\Users\ashle\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [7]:
# Load data for Ashley
data = pd.read_csv("C:/Users/ashle/OneDrive/Documents/Courses/UNC Courses/665 - Applied Stat II/2025_NBA_Project/nba_playoffdata_combined.csv")

In [8]:
# Save team name columns
winners = data['Winner'].copy()
losers = data['Loser'].copy()

# Drop unnecessary columns
data.drop(columns=['Winner', 'Loser', 'Winner Team'], inplace=True)

# Create net win column (response var)
def compute_net_wins(row):
    win_diff = row['Winner Games Won'] - row['Loser Games Won']

    if row['Winner Seed'] < row['Loser Seed']:
        return win_diff
    else:  
        return -win_diff

data['Net Wins'] = data.apply(compute_net_wins, axis=1)

In [12]:
# Check data
data

,Year,Round,Winner Games Won,Loser Games Won,Winner Seed,Loser Seed,Winner Rk,Winner MP,Winner FG,Winner FGA,...,Loser ORB,Loser DRB,Loser TRB,Loser AST,Loser STL,Loser BLK,Loser TOV,Loser PF,Loser PTS,Net Wins
0,2024,Eastern Conf First Round,4,1,1,8,2,241.8,43.9,90.2,...,9.3,33.0,42.3,25.8,7.5,3.4,12.7,17.3,110.1,3
1,2024,Eastern Conf First Round,4,3,4,5,20,241.5,41.8,87.2,...,10.5,31.8,42.3,24.7,8.2,5.2,14.7,19.7,110.5,1
2,2024,Eastern Conf First Round,4,2,6,3,1,240.3,47.0,92.7,...,9.4,34.8,44.2,26.5,6.8,5.0,12.9,19.2,119.0,-2
3,2024,Eastern Conf First Round,4,2,2,7,19,240.6,41.3,88.7,...,11.0,31.9,43.0,24.9,8.5,6.0,12.0,20.3,114.6,2
4,2024,Western Conf First Round,4,0,1,8,3,241.5,44.5,89.3,...,10.4,33.6,44.0,27.0,8.3,4.6,13.0,18.4,115.1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,1980,Western Conf Semifinals,4,1,1,4,2,242.4,47.5,89.9,...,13.1,30.0,43.0,27.8,11.1,4.2,19.9,22.6,111.1,3
652,1980,Western Conf Semifinals,4,3,3,2,12,241.8,43.3,92.3,...,15.2,29.2,44.4,27.8,9.5,6.2,18.2,23.6,110.1,-1
653,1980,Eastern Conf Finals,4,1,3,1,10,242.1,43.0,87.3,...,15.0,30.0,44.9,26.8,9.9,3.8,18.8,24.1,113.5,-3
654,1980,Western Conf Finals,4,1,1,3,2,242.4,47.5,89.9,...,16.8,31.1,47.9,24.9,9.1,5.2,18.2,22.7,108.5,3


In [16]:
# Train-test split
X = data.drop('Net Wins', axis=1)
y = data['Net Wins']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=665)

print(X_train.shape)
print(X_test.shape)

(459, 53)
(197, 53)
